<img src="/Users/sanyachauhan/Desktop/hackathon/1.png" width="20%" />

### Model Workflow

1. **Calendar Event's Information Extraction:** Using <span style="color:blue">Google Calendar API</span>, we extract information from Salesperson’s Google Calendar and auto-populated in a Google Sheet, everyday, for next day’s events.

2. **LinkedIn Profile's Link Extraction:** Using <span style="color:blue">Google Sheet API</span>, we use names and organisations of the prospective clients to find the LinkedIn profile link of the prospective client. The first link after the Google search is assumed to be the LinkedIn link. 

3. **Saving LinkedIn's Profile Data into Text Files:** The LinkedIn profile information (for salesperson and client), using <span style="color:blue">Web Search</span>, is saved as a text file.

4. **Response Generation using GenAI Model <span style="color:blue">Gemini Pro</span>:** A prompt with additiona information from attached files (LinkedIn profiles + past email conversations) are fed to a code using Gemini model. The respons is the required 5 and 1 minute summary.

5. **Automated Email Scheduling:** The response from Gemini is sent to the salesperson in an email 10 minuted before every meeting as per the Google Calendar.

# Step 1
### Google Calendar Event's Information Extraction

This part of the code is executed in <span style="color:blue">Google Sheet's Extention:</span> <span style="color:red">Apps Script</span>

In [ ]:
# Replace all # with // to convert them into comments for Apps Script

# This function retrieves events from a Google Calendar and stores them in a Google Sheets spreadsheet.

function myFunction() {
  # Access the currently active Google Spreadsheet
  var ss = SpreadsheetApp.getActiveSpreadsheet();

  # Try to get the sheet named "GetEvents" if it exists, otherwise create a new sheet with that name
  var sheet = ss.getSheetByName("GetEvents") || ss.insertSheet("GetEvents");

  # Access the Google Calendar with the specified ID
  var cal = CalendarApp.getCalendarById("mitwork100@gmail.com");

  # Get today's date
  var today = new Date();

  # Add 1 day to get tomorrow's date
  var tomorrow = new Date(today);
  tomorrow.setDate(tomorrow.getDate() + 1);

  # Set the start of tomorrow to 12:00 AM
  var startOfTomorrow = new Date(tomorrow.setHours(0, 0, 0, 0));

  # Set the end of tomorrow to 11:59 PM
  var endOfTomorrow = new Date(tomorrow.setHours(23, 59, 59, 999));

  # Retrieve events happening tomorrow from the Google Calendar
  var events = cal.getEvents(startOfTomorrow, endOfTomorrow);

  # Loop through each event retrieved
  for(var i = 0; i < events.length; i++){
    # Extract details of the event
    var title = events[i].getTitle();
    var start = events[i].getStartTime();
    var end = events[i].getEndTime();

    # Get the spreadsheet's time zone and format event start and end times accordingly
    var timeZone = ss.getSpreadsheetTimeZone();
    var start_time = Utilities.formatDate(start, timeZone, "HH:mm");
    var end_time = Utilities.formatDate(end, timeZone, "HH:mm");

    # Extract event description
    var des = events[i].getDescription();

    # Extract the email address of the event creator
    var creators = events[i].getCreators();

    # Extract the list of guests invited to the event
    var guests = events[i].getGuestList();
    var guestCount = guests.length;

    # Create an array to store guest email addresses
    var guestEmails = [];
    for (var j = 0; j < guests.length; j++) {      
      guestEmails.push(guests[j].getEmail());
    }

    # Join all emails (including the organizer's) into a single string separated by commas
    var client_email = guestEmails.join(", ");

    # Write event details to the spreadsheet
    sheet.getRange(i+2,1).setValue(title); // Write event title
    sheet.getRange(i+2,2).setValue(start_time); // Write event start time
    sheet.getRange(i+2,3).setValue(end_time); // Write event end time
    sheet.getRange(i+2,4).setValue(client_email); // Write guest emails
    sheet.getRange(i+2,5).setValue(creators); // Write event creators
    sheet.getRange(i+2,6).setValue(des); // Write event description
  }

  # Log a message indicating that events have been successfully added to the spreadsheet
  Logger.log("Events have been added to the Spreadsheet");
}